In [3]:
import numpy as np
import pandas as pd
import re

from collections import Counter
import string

from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.utils import pprint
from konlpy.tag import Twitter

import os

from tqdm import tqdm

from gensim.models import Word2Vec
import random

import time

import threading

In [4]:
kkma = Kkma()
twitter = Twitter()

C:\Users\Samsung\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


# data loading

In [5]:
# 3개의 데이터 있는 dir 지정하기
path_ = r"C:\Users\Samsung\jupyter\Project_Deep_learning_2020\data\ear_phone"
file_list = os.listdir(path_)
file_list_xlsx = [f for f in file_list if f.endswith(".xlsx")]
file_list_xlsx

['air.xlsx', 'air_pro.xlsx', 'buzz.xlsx']

In [6]:
for f in file_list_xlsx:
    file_path = path_ + "\\" + f
    name = f.replace(".xlsx","")
    globals () ["df_{}".format(name)] =  pd.read_excel(file_path).drop(columns = "Unnamed: 0")
    globals () ["df_{}".format(name)]["contents"] =  globals () ["df_{}".format(name)]["contents"].map(lambda x : str(x).replace("None",""))

# pos tagging

- version 2가지 중 twitter 사용 권장

In [7]:
def pos_tagging(df_,version):
    pos = []
    words = []
    for i in tqdm(range(len(df_))):
        if version == "kkma":
            p = kkma.pos(df_["contents"][i])
        elif version == "twitter":
            p = twitter.pos(df_["contents"][i])
        word = [w[0] for w in p]
        pos.append(p)
        words.append(word)
    df_["tag"] = pos
    df_["words"] = words
    return df_

In [8]:
test = df_air

,date,id,item,score,contents,liked,Cate
32215,2020.01.29,허*영,"Apple 에어팟 2세대 유선 충전 모델 (블루투스 5.0), MV7N2KH/A, ...",5,2020.01.29\n저는 삼성 갤럭시 S9+ 쓰는데 친구들이 다 에어팟을 들고 있...,52,air
32216,2020.01.29,신*진,"Apple 에어팟 2세대 유선 충전 모델 (블루투스 5.0), MV7N2KH/A, ...",5,,0,air
32217,2020.01.29,유*수,"Apple 에어팟 2세대 유선 충전 모델 (블루투스 5.0), MV7N2KH/A, ...",5,,0,air
32218,2020.01.29,성*수,"Apple 에어팟 2세대 유선 충전 모델 (블루투스 5.0), MV7N2KH/A, ...",1,아이폰이아니라서 끊겨서들림 업데이트해도 잘안됨,0,air
32219,2020.01.29,변*주,"Apple 에어팟 2세대 유선 충전 모델 (블루투스 5.0), MV7N2KH/A, ...",5,짱짱,0,air


In [ ]:
## test 에 자신 data 넣어서 돌리기

test = df_air
test_pos = pos_tagging(test,"twitter")

 46%|██████████████████████████████████▍                                        | 32196/70146 [01:00<00:49, 772.59it/s]

# 분석에 사용할 품사 선택을 위한 과정

In [ ]:
def pos_counter(df_tag):
    tag = []
    for i in range(len(df_tag)):
        for i in df_tag["tag"][i]:
            tag.append(i[1])
    return Counter(tag).keys(),Counter(tag)

In [ ]:
def tag_sear(df,pos):
    all_ = []
    wo = []
    for li in df["tag"]:
        for i in li:
            if i[1] == pos:
                all_.append(i)
                wo.append(i[0])
    return all_,wo

In [ ]:
tag_,counter_ = pos_counter(test_pos)

In [ ]:
for key in  tag_:
    print(key)
    print(Counter((tag_sear(test_pos,key))[1]).most_common(100))
    print("_"*40)

# 원하는 품사선택

In [ ]:
def only_(df,igs):
    list_1 = []
    for list_ in df.loc[:,"tag"]:
        list_2 = []
        for i in range(len(list_)):
            #print(list_[i])
            if list_[i][1] in igs:
                list_2.append(list_[i][0])
        list_1.append(list_2)
    return list_1

In [ ]:
po_list = ["Noun","Adjective","Verb","Adverb","Alpha"]
all_pos = tag_
customize = ["Noun","Adjective","Verb","Adverb","Alpha"]

In [ ]:
def split_tag(df_,pos_list,customize,pos_all):
    for tag in pos_list:
        df_.loc[:,tag] = only_(df_,tag)
    df_.loc[:,"customize"] = only_(df_,customize)
    df_.loc[:,"all"] = only_(df_,pos_all)
    return df_

In [ ]:
test_split = split_tag(test_pos,po_list,customize,all_pos)
test_split.head()

# word2vec model 만들기

In [ ]:
df_all = test_split
win_size = 5
#target_col = "All"

def word2vec_model(df_all,target_col,win_size):
    model_words = Word2Vec(sentences=df_all[target_col], window = win_size, min_count=0,iter=100, sg=1)
    return model_words

In [ ]:
model_word = word2vec_model(test_split,"all",5)
model_word.most_similar("좋아요")

In [ ]:
model_word = word2vec_model(test_split,"customize",5)
model_word.most_similar("좋아요")

# 가장 많이 나오는 단어 데이터 만들기

In [ ]:

def make_mostdf(df_,target_tag,most_n):
    df_most = pd.DataFrame()
    word = Counter(df_[target_tag].sum()).most_common(most_n)
    df_most["word"] = [n[0] for n in word]
    df_most["count"] = [n[1] for n in word]
    return df_most


In [ ]:
### 선택 parameter 상위 몇개 추출?
most_n = 800
target_tag = "Noun"
dm = make_mostdf(test_split,target_tag,most_n)
dm.head()

# 나중에 topic modeling를 쓰기 위해서 데이터 추출

- 해당 단어가 있는 sentence 추출하기

In [ ]:
def find_sentence(df,target,version):
    senten = []
    a = df.loc[:,version]
    for i in a:
        if target in i:
            senten.append(i)
    return senten

In [ ]:
def extract_(df,dm,version):
    sen_ = []
    for i in dm.loc[:,"word"]:
        a_ = find_sentence(df,i,version)
        sen_.append(a_)
    dm["sentences"] = sen_
    return dm

In [ ]:
dm_2 = extract_(test_split,dm,"customize")
dm_2.head()

# word2vec 결과도 같이
- 해당 단어랑 가장 비슷한 단어 산출

In [ ]:
def most_similar(df_,w2_model):
    most_simi = []
    for i in df_["word"]:
        ms = w2_model.wv.most_similar(i,topn=20)
        most_simi.append(ms)
    df_["word2vec"] = most_simi
    return df_

In [ ]:
df_m = most_similar(dm_2,model_word)
df_m.head()

# Topic modeling 도 같이

In [ ]:
def topic_moding(k,documents):
    K=k
    random.seed(0)
    D = len(documents)
    def p_topic_given_document(topic, d, alpha=0.1):
        return ((document_topic_counts[d][topic] + alpha) /(document_lengths[d] + K * alpha))

    def p_word_given_topic(word, topic, beta=0.1):
        return ((topic_word_counts[topic][word] + beta) /(topic_counts[topic] + V * beta))

    def topic_weight(d, word, k):
        return p_word_given_topic(word, k) * p_topic_given_document(k, d)

    def choose_new_topic(d, word):
        return sample_from([topic_weight(d, word, k) for k in range(K)])

    def sample_from(weights):
        total = sum(weights)
        rnd = total * random.random()
        for i, w in enumerate(weights):
            rnd -= w
            if rnd <= 0:
                return i

    document_topics = [[random.randrange(K) for word in document] for document in documents]
    document_topic_counts = [Counter() for _ in documents]
    topic_word_counts = [Counter() for _ in range(K)]
    topic_counts = [0 for _ in range(K)]
    document_lengths = list(map(len, documents))
    distinct_words = set(word for document in documents for word in document)
    V = len(distinct_words)

    D = len(documents)
    for d in range(D):
        for word, topic in zip(documents[d], document_topics[d]):
            document_topic_counts[d][topic] += 1
            topic_word_counts[topic][word] += 1
            topic_counts[topic] += 1


    for d in range(D):
        for word, topic in zip(documents[d], document_topics[d]):
            document_topic_counts[d][topic] += 1
            topic_word_counts[topic][word] += 1
            topic_counts[topic] += 1
    return document_topic_counts, topic_word_counts

In [ ]:
def topic_modeling_edit(data_,window):
    k = 1
    Topic_2 = []
    for num_ in data_.index:
        list_ = data_.loc[num_,"sentences"]
        word_ = data_.loc[num_,"word"]
        try:
            removal = [li.remove(word_) for li in list_]
        except:
            pass
        docu_,topic_words = topic_moding(k,list_)
        top = topic_words[0].most_common(window)
        Topic_2.append(top)
    data_["Topic"] = Topic_2
    return data_

In [ ]:
df_topic = topic_modeling_edit(df_m,10)

In [ ]:
df_topic.head()

In [ ]:
df_topic.to_excel(r"C:\Users\Samsung\jupyter\Project_Deep_learning_2020\data\ear_phone\most\air.xlsx")